In [1]:
# Importing the necessary packages 
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Layer,Lambda

In [3]:
# load dataset
forestfires = pd.read_csv("forestfires.csv")

In [4]:
#As dummy variables are already created, we will remove the month and also day columns 
forestfires.drop(["month","day"],axis=1,inplace = True)

In [5]:
forestfires["size_category"].value_counts() 

small    378
large    139
Name: size_category, dtype: int64

In [6]:
forestfires.isnull().sum()

FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [7]:
print(forestfires.describe())

             FFMC         DMC          DC         ISI        temp          RH  \
count  517.000000  517.000000  517.000000  517.000000  517.000000  517.000000   
mean    90.644681  110.872340  547.940039    9.021663   18.889168   44.288201   
std      5.520111   64.046482  248.066192    4.559477    5.806625   16.317469   
min     18.700000    1.100000    7.900000    0.000000    2.200000   15.000000   
25%     90.200000   68.600000  437.700000    6.500000   15.500000   33.000000   
50%     91.600000  108.300000  664.200000    8.400000   19.300000   42.000000   
75%     92.900000  142.400000  713.900000   10.800000   22.800000   53.000000   
max     96.200000  291.300000  860.600000   56.100000   33.300000  100.000000   

             wind        rain         area      dayfri  ...    monthdec  \
count  517.000000  517.000000   517.000000  517.000000  ...  517.000000   
mean     4.017602    0.021663    12.847292    0.164410  ...    0.017408   
std      1.791653    0.295959    63.655818   

In [8]:
# I am taking small as 0 and large as 1
forestfires.loc[forestfires["size_category"]=='small','size_category']=0 
forestfires.loc[forestfires["size_category"]=='large','size_category']=1 
forestfires["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [9]:
#Normalization being done. 
def norm_func(i):
    x = (i-i.min()) / (i.max() - i.min()) 
    return (x)

In [10]:
predictors= forestfires.iloc[:,0:28] 
target= forestfires.iloc[:,28]

In [13]:
predictors1 = norm_func(predictors)
#data= pd.concat([predictors1,target],axis=1)

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(predictors1,target, test_size=0.3,stratify = target)

In [22]:
#Creating base model
def prep_model(hidden_dim): 
    model= Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu")) 
        else:
            model.add(Dense(hidden_dim[i],activation="relu")) 
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"]) 
    return model

In [23]:
first_model= prep_model([28,50,40,20,1])
first_model.fit((np.asarray(x_train).astype(np.int)),(np.asarray(y_train).astype(np.int)),epochs=500) 
pred_train = first_model.predict(x_train)

Epoch 1/500
12/12 [==============================] - 29s 8ms/step - loss: 0.6733 - accuracy: 0.7343
Epoch 2/500
12/12 [==============================] - 0s 3ms/step - loss: 0.6276 - accuracy: 0.7298
Epoch 3/500
12/12 [==============================] - 0s 3ms/step - loss: 0.5812 - accuracy: 0.7553
Epoch 4/500
12/12 [==============================] - 0s 4ms/step - loss: 0.5801 - accuracy: 0.7322
Epoch 5/500
12/12 [==============================] - 0s 3ms/step - loss: 0.5832 - accuracy: 0.7234
Epoch 6/500
12/12 [==============================] - 0s 3ms/step - loss: 0.5451 - accuracy: 0.7610
Epoch 7/500
12/12 [==============================] - 0s 3ms/step - loss: 0.5663 - accuracy: 0.7432
Epoch 8/500
12/12 [==============================] - 0s 3ms/step - loss: 0.5812 - accuracy: 0.7276
Epoch 9/500
12/12 [==============================] - 0s 3ms/step - loss: 0.6033 - accuracy: 0.6988
Epoch 10/500
12/12 [==============================] - ETA: 0s - loss: 0.6163 - accuracy: 0.71 - 0s 3ms/step 

In [24]:
#Converting the predicted values to series
pred_train = pd.Series([i[0] for i in pred_train])

In [25]:
size = ["small", "large"]
pred_train_class = pd.Series(["small"]*361)
pred_train_class[[i>0.5 for i in pred_train]]= "large"

In [26]:
train = pd.concat([x_train, y_train],axis=1)
train["size_category"].value_counts()

0    264
1     97
Name: size_category, dtype: int64

In [27]:
#For training data
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["size_category"]==1,"original_class"] = "large" 
train.original_class.value_counts()

small    264
large     97
Name: original_class, dtype: int64

In [28]:
confusion_matrix(pred_train_class,train["original_class"]) 

array([[  0,   0],
       [ 97, 264]], dtype=int64)

In [29]:
np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True)) 

0.7313019390581718

In [30]:
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
small,97,264


In [31]:
#For test data
pred_test = first_model.predict(np.array(x_test)) 
pred_test = pd.Series([i[0] for i in pred_test])

In [32]:
pred_test_class = pd.Series(["small"]*156) 
pred_test_class[[i>0.5 for i in pred_test]] = "large"

In [35]:
test =pd.concat([x_test,y_test],axis=1) 
test["original_class"]="small" 
test.loc[test["size_category"]==1,"original_class"] = "large"
test["original_class"].value_counts() 

small    114
large     42
Name: original_class, dtype: int64

In [36]:
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True))

0.7307692307692307

In [37]:
confusion_matrix(pred_test_class,test["original_class"])

array([[  0,   0],
       [ 42, 114]], dtype=int64)

In [38]:
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
small,42,114
